# Методы оптимизации Pandas

In [14]:
%reload_ext memory_profiler

## Зачем нужна оптимизация в Pandas

Хотя Pandas удобен и гибок, при работе с большими объёмами данных он может потреблять очень много оперативной памяти и работать медленно. Особенно это проявляется при:
- загрузке данных с миллионами строк,
- множественных группировках и агрегациях,
- использовании `apply` или итераций по строкам.

### Оптимизация позволяет:
- 📈 **Повысить производительность** — сократить время выполнения операций, особенно при работе с большими наборами данных.
- 💾 **Снизить потребление памяти** — уменьшить объём данных за счёт оптимальных типов (`category`, `int8`, `float32`).
- 🧠 **Сделать код читаемым и поддерживаемым** — за счёт использования векторных операций и встроенных методов Pandas вместо универсальных, но медленных решений (как `apply`).

В этом практическом блокноте мы рассмотрим ключевые подходы к оптимизации, сравним производительность разных подходов, и научимся обрабатывать большие объёмы данных эффективно.


## 1. Оптимизация по памяти

### В чём суть

По умолчанию при создании `DataFrame` из NumPy-массивов или при чтении из CSV-файлов Pandas выбирает **тип данных (dtype)** автоматически. Однако эти типы часто бывают избыточными:

- `int64` используется даже для колонок, где максимум 0–100.
- `float64` применяется к вещественным числам, где хватит `float32`.
- `object` используется для строк и категориальных значений, хотя можно применить `category`.

Это приводит к **чрезмерному расходу оперативной памяти**, особенно при работе с миллионами строк.

---

### Как экономить память

#### 🔹 Использовать `astype()` для приведения типов:

```python
df['column'] = df['column'].astype('тип')


### Полезные типы для оптимизации

✅ **`category`**  
Подходит для колонок с ограниченным числом уникальных значений (например, пол, страна, метки).  

- Экономит память: строка `"Россия"` хранится один раз и ссылается по индексу.
- Ускоряет `groupby`, `merge` и фильтрацию.

```python
df['gender'] = df['gender'].astype('category')


✅ **`int8`, `int16`, `int32` вместо `int64`**  
Если значения укладываются, можно использовать меньший тип:

- `int8`: от -128 до 127  
- `uint8`: от 0 до 255

```python
df['age'] = df['age'].astype('int8')


✅ **`float32` вместо `float64`**  
В два раза меньше памяти, если не требуется высокая точность.

```python
df['score'] = df['score'].astype('float32')


### Как измерить экономию

Используем `df.info(memory_usage='deep')` до и после оптимизации, чтобы увидеть точный объём памяти, занимаемый DataFrame.

```python
print(df.info(memory_usage='deep'))


### Когда **НЕ** стоит оптимизировать

- Не стоит понижать тип чисел, если требуется высокая точность (например, в научных расчётах).
- Не стоит использовать `category`, если в колонке слишком много уникальных значений — это может даже увеличить использование памяти.

---

### Вывод

Понижение типа данных — один из самых простых и эффективных способов оптимизации. Особенно важно при:

- загрузке CSV-файлов в память,
- работе с миллионами строк,
- подготовке данных к машинному обучению.


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time


In [4]:
"""
Задача:
Показать, как сократить использование оперативной памяти за счёт изменения типов данных.
"""

n = 1_000_000
data = {
    'id': np.arange(n),
    'category': np.random.choice(['A', 'B', 'C', 'D'], size=n),
    'value': np.random.rand(n) * 100,
    'flag': np.random.choice([0, 1], size=n),
    'text': np.random.choice(['foo', 'bar', 'baz'], size=n)
}

print("До оптимизации типов:")
df = pd.DataFrame(data)
print(df.info(memory_usage='deep'))

# Оптимизация типов
optimized_df = df.copy()
optimized_df['category'] = optimized_df['category'].astype('category')
optimized_df['flag'] = optimized_df['flag'].astype('bool')
optimized_df['text'] = optimized_df['text'].astype('category')
optimized_df['value'] = optimized_df['value'].astype('float32')

print("\nПосле оптимизации типов:")
print(optimized_df.info(memory_usage='deep'))



До оптимизации типов:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 5 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   id        1000000 non-null  int32  
 1   category  1000000 non-null  object 
 2   value     1000000 non-null  float64
 3   flag      1000000 non-null  int32  
 4   text      1000000 non-null  object 
dtypes: float64(1), int32(2), object(2)
memory usage: 127.8 MB
None

После оптимизации типов:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 5 columns):
 #   Column    Non-Null Count    Dtype   
---  ------    --------------    -----   
 0   id        1000000 non-null  int32   
 1   category  1000000 non-null  category
 2   value     1000000 non-null  float32 
 3   flag      1000000 non-null  bool    
 4   text      1000000 non-null  category
dtypes: bool(1), category(2), float32(1), int32(1)
memory usage: 10.5 MB
None


In [9]:
df.to_csv("large_data.csv")

## 🧠 2. Векторизация против apply — почему это важно

### ❓ Пример задачи:

Нужно создать новый столбец `label`, в котором будет:

- `'high'`, если значение в `value > 50`;
- `'low'` — в остальных случаях.

Можно сделать это через `apply`, но гораздо эффективнее — через NumPy-функции вроде `np.where`.


### 🔧 NumPy — это низкоуровневая библиотека, Pandas — высокоуровневая обёртка

**NumPy** — это библиотека для работы с массивами, написанная на языке **C**.  
Она умеет обрабатывать **целые массивы данных как единое целое**, используя быстрые циклы на уровне процессора.

**Pandas** — это библиотека, написанная на **Python**, которая строится **поверх NumPy**.  
Она предоставляет более удобный интерфейс (таблицы, метки, методы вроде `apply`, `groupby`), но под капотом всё равно использует массивы NumPy.

---

📌 **Пример аналогии:**

- NumPy — это **двигатель гоночной машины**.  
- Pandas — это **машина с рулём, панелью и навигатором**, которая на этом двигателе едет.


### 🔁 Почему `apply` в Pandas медленный

Когда производится операция `df.apply(func, axis=1)`:

- для **каждой строки** Pandas:
  - создаёт отдельный объект `Series` (это как мини-таблица),
  - вызывает твою Python-функцию,
  - сохраняет результат обратно в новый столбец.

➡️ Это **миллион вызовов Python-функции** и **миллион объектов `Series`**.

❗ Всё это — работа **интерпретатора Python**, а не компилированного кода. Поэтому `apply` работает **медленно**, особенно на больших таблицах.


### ⚡ Почему NumPy и векторизация работают быстрее

NumPy-функции (`np.where`, `np.select`, логические маски и т.д.):

- работают на уровне **массивов (`ndarray`)**, без создания отдельных объектов на каждую строку;
- написаны на **C или Fortran**, то есть выполняются на **нативной скорости**;
- **не требуют вызова Python-функции** для каждой строки;
- используют **SIMD-инструкции процессора**, если они доступны (это даёт ускорение на уровне «железа»).

---

**Пример:**

```python
df['label'] = np.where(df['value'] > 50, 'high', 'low')

Эта строка обрабатывает весь столбец сразу, а не по одной строке из миллиона. Это делает векторизацию в десятки раз быстрее, чем apply.


In [23]:
%%memit
"""
Задача:
Сравнить производительность и подход к решению задачи с помощью apply и векторизации (np.where).
"""
import time

def with_apply(df):
    return df.apply(lambda row: 'high' if row['value'] > 50 else 'low', axis=1)

def with_vectorized(df):
    return np.where(df['value'] > 50, 'high', 'low')

start = time.time()
df['label_apply'] = with_apply(df)
print("Apply time:", time.time() - start)

start = time.time()
df['label_vector'] = with_vectorized(df)
print("Vectorized time:", time.time() - start)

print("Equal results:", (df['label_apply'] == df['label_vector']).all())

Apply time: 8.644313335418701
Vectorized time: 0.0998387336730957
Equal results: True
peak memory: 647.13 MiB, increment: 234.95 MiB


## 3. Оптимизация группировок

### Проблема

Группировка данных — одна из самых частых операций в Pandas, особенно при агрегации (сумма, среднее, максимум и т.д.).  
Многие используют `groupby().apply()` для этих целей, но это **медленный и затратный** способ, особенно на больших таблицах.

---

### ❌ Почему `groupby().apply()` медленный

```python
df.groupby('category').apply(lambda x: x['value'].mean())


Внутри `apply`:

- создаются временные подтаблицы для каждой группы;
- вызывается Python-функция `lambda`;
- Pandas обрабатывает результат построчно.

➡ Всё это происходит в Python — медленно и с большими накладными расходами.

---

### ✅ Почему `groupby().agg()` быстрее

```python
df.groupby('category').agg(mean_value=('value', 'mean'))


- `agg()` не вызывает Python-функцию;
- использует **встроенные агрегирующие функции** (`mean`, `sum`, `count`, `min`, `max`);
- реализован на **C**, а значит работает **значительно быстрее**;
- результат **читаемый и компактный**, особенно при нескольких агрегатах.

---

### 🧪 Пример: среднее и сумма по группам

```python
df.groupby('region').agg(
    avg_salary=('salary', 'mean'),
    total_bonus=('bonus', 'sum')
)


In [18]:
%%time
%%memit
"""
Задача:
Показать, как использовать groupby().agg() для быстрого и читаемого агрегирования.
"""

# Пример с apply
result_apply = optimized_df.groupby('category').apply(lambda group: pd.Series({
    'mean_value': group['value'].mean(),
    'total_flag': group['flag'].sum()
}))


peak memory: 410.62 MiB, increment: 0.00 MiB
CPU times: total: 172 ms
Wall time: 1.27 s


<string>:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
<string>:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.


In [21]:
%%time
%%memit
# Пример с agg
result_agg = optimized_df.groupby('category').agg(
    mean_value=('value', 'mean'),
    total_flag=('flag', 'sum')
)


peak memory: 410.43 MiB, increment: 0.01 MiB
CPU times: total: 109 ms
Wall time: 1.24 s


<string>:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.


## 4. Работа с большими файлами

### 🧠 Проблема

Когда вы работаете с большими CSV-файлами (100 МБ, 500 МБ, 1+ ГБ), попытка прочитать их **целиком в память** с помощью `pd.read_csv()` может:

- сильно замедлить выполнение;
- привести к **Out of Memory** (особенно на слабых машинах);
- затруднить анализ и обработку из-за перегрузки оперативки.

---

### 🛠 Решение — `chunksize` в `read_csv`

Метод `pd.read_csv()` поддерживает параметр `chunksize`, который позволяет:

- **не загружать весь файл сразу**;
- **обрабатывать данные частями** — например, по 100 000 строк за раз;
- **экономить оперативную память** и повышать стабильность работы скрипта.

```python
reader = pd.read_csv('data.csv', chunksize=100_000)


### 🔁 Как это работает

`read_csv(..., chunksize=...)` возвращает **генератор**, который выдаёт DataFrame-подобные части файла по указанному количеству строк.

Это позволяет вам:

- выполнять агрегирование, фильтрацию, подсчёты **постепенно**;
- сохранять **промежуточные результаты**;
- **объединять** их в конце, если нужно.

---

### 🧪 Пример: посчитать среднее значение по категориям

```python
chunk_results = []

for chunk in pd.read_csv('data.csv', chunksize=100_000):
    summary = chunk.groupby('category')['value'].mean()
    chunk_results.append(summary)

final_result = pd.concat(chunk_results).groupby(level=0).mean()


### 💡 Как это помогает

- Загрузка больших файлов **без перегрузки оперативной памяти**.
- Можно делать **пайплайн обработки по частям** — фильтрация, очистка, агрегация.
- Идеально подходит для:
  - **ETL-процессов** (извлечение-преобразование-загрузка),
  - **предварительной обработки данных**,
  - **парсинга логов** и больших CSV/TSV файлов.


In [ ]:
%load_ext memory_profiler

In [15]:
%%memit
"""
Задача:
Показать, как читать и обрабатывать большие CSV-файлы по частям с помощью chunksize.
"""

# ✅ Подход 1: чтение целиком
start_full = time.time()
df_full = pd.read_csv('large_data.csv')
full_result = df_full.groupby('category')['value'].mean()
end_full = time.time()

print("📦 Полное чтение и группировка:")
print(full_result)
print(f"⏱ Время (целиком): {end_full - start_full:.4f} сек")

📦 Полное чтение и группировка:
category
A    50.090653
B    49.958815
C    50.020008
D    50.018708
Name: value, dtype: float64
⏱ Время (целиком): 1.3574 сек
peak memory: 601.06 MiB, increment: 190.41 MiB


In [17]:
%%memit
# 🔁 Подход 2: чтение по чанкам
start_chunks = time.time()
chunk_results = []
for chunk in pd.read_csv('large_data.csv', chunksize=200_000):
    summary = chunk.groupby('category')['value'].mean()
    chunk_results.append(summary)

chunk_result = pd.concat(chunk_results).groupby(level=0).mean()
end_chunks = time.time()

print("\n🔃 Чтение по частям и группировка:")
print(chunk_result)
print(f"⏱ Время (чанки): {end_chunks - start_chunks:.4f} сек")


🔃 Чтение по частям и группировка:
category
A    50.090615
B    49.958811
C    50.019465
D    50.018582
Name: value, dtype: float64
⏱ Время (чанки): 1.4302 сек
peak memory: 436.32 MiB, increment: 25.66 MiB


## 5. Профилирование

Когда мы оптимизируем код на Pandas, важно понимать:
- сколько **времени** занимает операция;
- сколько **памяти** она потребляет.

Для этого в Jupyter есть удобные **магические команды** и сторонние инструменты.

---

### ⏱ Время выполнения

#### ✅ `%%time` — замер времени выполнения всей ячейки

```python
%%time
df.groupby('category')['value'].mean()


Покажет общее время выполнения в формате **Wall time**.

---

✅ `%timeit` — более точное усреднение времени

```python
%timeit df['value'] > 50


#### ✅ `%time` — замер времени выполнения одной строки

```python
%time df.groupby('category')['value'].mean()

### 💾 Замер использования памяти

✅ **Установка `memory_profiler`**

```python
!pip install -q memory_profiler


✅ **Подключение в Jupyter**

```python
%load_ext memory_profiler


✅ **Использование `%memit`**

```python
%%memit
df['label'] = np.where(df['value'] > 50, 'high', 'low')


`%memit` покажет **пиковое потребление памяти в мегабайтах** во время выполнения кода.
